# Import libraries & features

In [ ]:
# You need to restart runtime after this step !!!
!pip uninstall python-louvain
!pip install python-louvain

Found existing installation: python-louvain 0.16
Uninstalling python-louvain-0.16:
  Would remove:
    /usr/local/bin/community
    /usr/local/lib/python3.7/dist-packages/community/*
    /usr/local/lib/python3.7/dist-packages/python_louvain-0.16.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/community/app.py
Proceed (y/n)? y
  Successfully uninstalled python-louvain-0.16
     |████████████████████████████████| 204 kB 4.4 MB/s 
  Created wheel for python-louvain: filename=python_louvain-0.16-py3-none-any.whl size=9408 sha256=6179e84442d95c748770a9f3f810a129941bbbb01b6f614bbac732716ea8a7db
  Stored in directory: /root/.cache/pip/wheels/f6/23/43/a714ed84811240ec3e8e709b8594ecdfbc8ad36ca5d949e38e
Successfully built python-louvain


In [ ]:
import pandas as pd
import numpy as np
import os
import nltk
import pickle
from time import time
import networkx as nx
import sklearn.preprocessing as preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.tree import DecisionTreeClassifier

from utils import WordEmbedding, GraphStructure, nbr_common_authors, compute_intersection, list_of_publication_2, \
    compute_affinity_between_authors, make_feature_vector, put_authors_in_dict, stack_lists



# Compute Features

In [ ]:
from sklearn.model_selection import train_test_split
from compute_features import ComputeFeatures

from sklearn.preprocessing import scale

c = ComputeFeatures()

Loading node information...
Loading train array...
Loading test array...
Loading stemmer and stop words...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
TfidVectorizer...
Creating word embeddings...
Training wv model with standard params...
Creating publication column...
Creating publication II column...
Creating authors dictionary...
Making feature vectors for the abstract...
Making feature vectors for the title...
Making graph structure...
Loaded betweenness dictionary!
Loaded closeness dictionary!
Loaded clustering_coeff dictionary!
Loaded degree dictionary!
Loaded eigenvector dictionary!
Loaded pagerank dictionary!
Loaded partition file.
Add Community Attributes


If you want to train your model only using specific variables, you just need to do:

my_var = ["l2_distance", "common_neighbors", "degree"]

X_tot =  c.compute_multiple_variables(my_var, train=True, scale=False)

In [ ]:
add_featrues = ['overlap_title', 'title_tdidf', 'temp_diff','comm_auth', 'overlap_abstract', 'abstract_tdidf', 'same_journal', 'same_child', 'same_parent', \
                        'nb_source_childs', 'nb_target_childs', 'nb_source_parents', 'nb_target_parents']
add_var = ['title_tdidf', 'overlap_abstract', 'same_child', 'same_parent', 'nb_source_childs', 'nb_target_childs', 'nb_source_parents', 'nb_target_parents']

c.handled_variables = c.handled_variables + add_var

In [ ]:
c.handled_variables

['publication_2',
 'adam_coeff',
 'overlapping_words_in_title',
 'number_of_common_authors',
 'difference_of_years',
 'affinity_between_authors',
 'identical_journal',
 'l2_distance',
 'cosine_distance_tfid',
 'l2_distance_between_titles',
 'common_neighbors',
 'clustering_coeff',
 'betweenness',
 'closeness',
 'degree',
 'eigenvector',
 'jaccard_coeff',
 'shortest_path',
 'pagerank',
 'community',
 'lp_within_inter_cluster',
 'lp_ra_index_soundarajan',
 'lp_cn_soundarajan',
 'lp_preferential_attachment',
 'lp_resource_allocation_index',
 'title_tdidf',
 'overlap_abstract',
 'same_child',
 'same_parent',
 'nb_source_childs',
 'nb_target_childs',
 'nb_source_parents',
 'nb_target_parents']

In [ ]:
len(c.handled_variables)

33

In [1]:
my_var = ['publication_2',
 'adam_coeff',
 'overlapping_words_in_title',
 'number_of_common_authors',
 #'difference_of_years',
 'affinity_between_authors',
 #'identical_journal',
 'l2_distance',
 #'cosine_distance_tfid',
 'l2_distance_between_titles',
 #'common_neighbors',
 'clustering_coeff',
 'betweenness',
 'closeness',
 'degree',
 'eigenvector',
 'jaccard_coeff',
 'shortest_path',
 'pagerank',
 'community',
 #'lp_within_inter_cluster',
 'lp_ra_index_soundarajan',
 #'lp_cn_soundarajan',
 'lp_preferential_attachment',
 #'lp_resource_allocation_index',
 #'title_tdidf',
 'overlap_abstract',
 'same_child',
 #'same_parent',
 #'nb_source_childs',
 'nb_target_childs',
 #'nb_source_parents',
 #'nb_target_parents'
]


# Dimensionality Reduction

In [ ]:
from sklearn.decomposition import PCA
X_tot = c.compute_multiple_variables("all", train=True, scale=False)
X_test = c.compute_multiple_variables("all", train=False, scale=True)
print('we are using %d features in total'%X_tot.shape[1])



Loading STANDARD publication_2 file!
Loading STANDARD adam_coeff file!
Loading STANDARD overlapping_words_in_title file!
Loading STANDARD number_of_common_authors file!
Loading STANDARD difference_of_years file!
Loading STANDARD affinity_between_authors file!
Loading STANDARD identical_journal file!
Loading STANDARD l2_distance file!
Loading STANDARD cosine_distance_tfid file!
Loading STANDARD l2_distance_between_titles file!
Loading STANDARD common_neighbors file!
Loading STANDARD clustering_coeff file!
Loading STANDARD betweenness file!
Loading STANDARD closeness file!
Loading STANDARD degree file!
Loading STANDARD eigenvector file!
Loading STANDARD jaccard_coeff file!
Loading STANDARD shortest_path file!
Loading STANDARD pagerank file!
Loading STANDARD community file!
Loading STANDARD lp_within_inter_cluster file!
Loading STANDARD lp_ra_index_soundarajan file!
Loading STANDARD lp_cn_soundarajan file!
Loading STANDARD lp_preferential_attachment file!
Loading STANDARD lp_resource_allo

In [ ]:
# Dimensionality reduction
pca = PCA(n_components=0.9, svd_solver='full')
pca.fit(X_tot)
X_tot_pca = pca.transform(X_tot)
X_test_pca = pca.transform(X_test)
print('after dimensionality reduction, we are using %d number of features '%X_tot_pca.shape[1])

after dimensionality reduction, we are using 15 number of features 


Using *X_tot_pca* as the first argument of *train_test_split()* for the results of dimensionality reduction. Otherwise, simply replace it with *X_tot*.

In [ ]:
# train_test split
X_train, X_validation, y_train, y_validation = train_test_split(X_tot, c.train_array[:, 2],stratify = c.train_array[:, 2]) # here, we used c.train_array[:, 2], the label list in the train set as the argument for stratification
# stratification means that the train_test_split method returns training and test subsets that have the same proportions of class labels as the input dataset
X_train = scale(X_train)
X_validation = scale(X_validation)
X_tot = scale(X_tot)
y_tot = c.train_array[:, 2]

In [ ]:
def decrease_of_acc(X, Y, names):
    names = c.handled_variables

    rf = RandomForestClassifier()
    shuffle_split = ShuffleSplit(n_splits=10)

    scores = defaultdict(list)

    # crossvalidate the scores on a number of different random splits of the data
    for train_idx, test_idx in shuffle_split.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        Y_train, Y_test = Y[train_idx], Y[test_idx]
        r = rf.fit(X_train, Y_train)
        acc = f1_score(Y_test, rf.predict(X_test))
        for i in range(X.shape[1]):
            X_t = X_test.copy()
            np.random.shuffle(X_t[:, i])
            shuff_acc = f1_score(Y_test, rf.predict(X_t))
            scores[names[i]].append((acc - shuff_acc) / acc)
    print("Features sorted by their score:")
    print(sorted([(round(np.mean(score), 4), feat) for feat, score in scores.items()], reverse=True))
    return scores

In [ ]:
# Get feature importance through decrease of accruacy if the feature is abandoned
decrease_of_acc(X_tot, y_tot, "all")

# Define metric print functions

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
def metrics_print(y_validation, y_pred):
    print("F1_score :  %4.3f \n"
          "Accuracy :  %4.3f \n"
          "Precision : %4.3f \n"
          "Recall :    %4.3f \n" % (
              f1_score(y_validation, y_pred), accuracy_score(y_validation, y_pred),
              precision_score(y_validation, y_pred),
              recall_score(y_validation, y_pred)))

# Define training algorithms

## Best performance: svm

In [ ]:
# Import algorithms
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
# from keras.models import Sequential
# from keras.layers import Dense, Dropout
# from keras.callbacks import EarlyStopping
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score
from collections import defaultdict
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def svc(X_train, y_train, X_validation, y_validation):
    print("SVM")

    model = SVC()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validation)
    metrics_print(y_validation, y_pred)
    return model

In [ ]:
svc(X_train, y_train, X_validation, y_validation)

SVM
F1_score :  0.984 
Accuracy :  0.982 
Precision : 0.985 
Recall :    0.983 



SVC()

In [ ]:
#hyper maramter tuning of SVM
from sklearn.model_selection import GridSearchCV
 
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True,scoring='f1', verbose = 3, cv = 3)
 
# fitting the model for grid search
grid.fit(X_train, y_train)

# print best parameter after tuning
print(grid.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

grid_predictions = grid.predict(X_validation)

# print classification report
metrics_print(y_validation, grid_predictions)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


## Other algorithms

Random Forest

In [ ]:
def randomforest(X_train, y_train, X_validation, y_validation):
    print("RF")
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validation)
    metrics_print(y_validation, y_pred)
    return model

In [ ]:
randomforest(X_train, y_train, X_validation, y_validation)

RF
F1_score :  0.978 
Accuracy :  0.976 
Precision : 0.974 
Recall :    0.981 



RandomForestClassifier()

In [ ]:
#hyper maramter tuning of Random Forest
from sklearn.model_selection import GridSearchCV
 
# defining parameter range
param_grid = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
 
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True,scoring='f1', verbose = 3)
 
# fitting the model for grid search
grid.fit(X_train, y_train)

# print best parameter after tuning
print(grid.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

grid_predictions = grid.predict(X_validation)

# print classification report
metrics_print(y_validation, grid_predictions)

Fitting 5 folds for each of 3960 candidates, totalling 19800 fits
[CV 1/5] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.982 total time= 4.0min
[CV 2/5] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.982 total time= 4.0min
[CV 3/5] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.982 total time= 3.9min
[CV 4/5] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.982 total time= 4.0min
[CV 5/5] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.983 total time= 3.9min
[CV 1/5] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=400;, score=0.982 total time= 7.8min
[CV 2/5] END boo

XGBoost

In [ ]:
from xgboost import XGBClassifier
def xgb(X_train, y_train, X_validation, y_validation):
    print("XGBoost")
    model = XGBClassifier(colsample_bytree=0.6, gamma=0.5, max_depth=5, min_child_weight=5, subsample=0.8)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validation)
    metrics_print(y_validation, y_pred)
    return model

In [ ]:
xgb(X_train, y_train, X_validation, y_validation)

XGBoost
F1_score :  0.983 
Accuracy :  0.982 
Precision : 0.987 
Recall :    0.980 



XGBClassifier(colsample_bytree=0.6, gamma=0.5, max_depth=5, min_child_weight=5,
              subsample=0.8)

In [ ]:
#hyper maramter tuning of XGBoost
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
 
# defining parameter range
param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
 
grid = GridSearchCV(XGBClassifier(learning_rate=0.02, n_estimators=600), param_grid, refit = True, verbose = 3, cv=3,scoring='f1')
 
# fitting the model for grid search
grid.fit(X_train, y_train)

# print best parameter after tuning
print(grid.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

grid_predictions = grid.predict(X_validation)

# print classification report
metrics_print(y_validation, grid_predictions)

Fitting 3 folds for each of 405 candidates, totalling 1215 fits
[CV 1/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.985 total time= 3.4min
[CV 2/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.985 total time= 3.4min
[CV 3/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.986 total time= 3.4min
[CV 1/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.8;, score=0.985 total time= 3.3min
[CV 2/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.8;, score=0.985 total time= 3.3min
[CV 3/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.8;, score=0.986 total time= 3.4min
[CV 1/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=1.0;, score=0.985 total time= 3.2min
[CV 2/3] END colsample_bytree=0.6, gamma=0.5, max_dept

KeyboardInterrupt: ignored

In [ ]:
# print best parameter after tuning
print(grid.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

grid_predictions = grid.predict(X_validation)

# print classification report
metrics_print(y_validation, grid_predictions)

Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
def ada(X_train, y_train, X_validation, y_validation):
    print("AdaBoost")
    model = AdaBoostClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validation)
    metrics_print(y_validation, y_pred)
    return model

In [ ]:
ada(X_train, y_train, X_validation, y_validation)

AdaBoost
F1_score :  0.976 
Accuracy :  0.974 
Precision : 0.987 
Recall :    0.964 



AdaBoostClassifier()

In [ ]:
#hyper maramter tuning of Adaboost
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
 
# defining parameter range
param_grid = {'base_estimator__max_depth':[i for i in range(2,11,2)],
              'base_estimator__min_samples_leaf':[5,10],
              'n_estimators':[10,50,250,1000],
              'learning_rate':[0.01,0.1]}
 
grid = GridSearchCV(AdaBoostClassifier(base_estimator=DecisionTreeClassifier()), param_grid, refit = True, verbose = 3, cv=3)
 
# fitting the model for grid search
grid.fit(X_train, y_train)

# print best parameter after tuning
print(grid.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

grid_predictions = grid.predict(X_validation)

# print classification report
metrics_print(y_validation, grid_predictions)

Decision Trees

In [ ]:
#decision Trees
def dt(X_train, y_train, X_validation, y_validation):
    print("Decision Trees")
    model = DecisionTreeClassifier(random_state=0,splitter='best', criterion='gini', max_depth=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validation)
    metrics_print(y_validation, y_pred)
    return model

In [ ]:
dt(X_train, y_train, X_validation, y_validation)

Decision Trees
F1_score :  0.965 
Accuracy :  0.962 
Precision : 0.972 
Recall :    0.957 



DecisionTreeClassifier(max_depth=42, random_state=0)

In [ ]:
#hyper maramter tuning of Adaboost
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
 
# defining parameter range
param_grid = {'criterion':['gini','entropy'],
              'max_depth':[10,11,12,15,20,30,40,50,70,90,120,150]}
 
grid = GridSearchCV(DecisionTreeClassifier(random_state=0,splitter='best'), param_grid, refit = True, verbose = 3, cv=3)
 
# fitting the model for grid search
grid.fit(X_train, y_train)

# print best parameter after tuning
print(grid.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

grid_predictions = grid.predict(X_validation)

# print classification report
metrics_print(y_validation, grid_predictions)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END ......criterion=gini, max_depth=10;, score=0.981 total time=   7.1s
[CV 2/3] END ......criterion=gini, max_depth=10;, score=0.982 total time=   6.9s
[CV 3/3] END ......criterion=gini, max_depth=10;, score=0.981 total time=   6.9s
[CV 1/3] END ......criterion=gini, max_depth=11;, score=0.981 total time=   7.6s
[CV 2/3] END ......criterion=gini, max_depth=11;, score=0.982 total time=   7.7s
[CV 3/3] END ......criterion=gini, max_depth=11;, score=0.981 total time=   7.8s
[CV 1/3] END ......criterion=gini, max_depth=12;, score=0.981 total time=   8.6s
[CV 2/3] END ......criterion=gini, max_depth=12;, score=0.981 total time=   8.2s
[CV 3/3] END ......criterion=gini, max_depth=12;, score=0.981 total time=   8.2s
[CV 1/3] END ......criterion=gini, max_depth=15;, score=0.980 total time=   9.9s
[CV 2/3] END ......criterion=gini, max_depth=15;, score=0.981 total time=  10.2s
[CV 3/3] END ......criterion=gini, max_depth=15;

Gradient Boosting

In [ ]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
def gbc(X_train, y_train, X_validation, y_validation):
    print("Gradient Boosting")
    model = GradientBoostingClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validation)
    metrics_print(y_validation, y_pred)
    return model

In [ ]:
gbc(X_train, y_train, X_validation, y_validation)

Gradient Boosting
F1_score :  0.980 
Accuracy :  0.979 
Precision : 0.988 
Recall :    0.973 



GradientBoostingClassifier()

In [ ]:
#hyper maramter tuning of Adaboost
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
 
# defining parameter range
param_grid = {
    "loss":["deviance"],
    "learning_rate": [0.05, 0.075, 0.1, 0.15, 0.2],
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "n_estimators":[10]
}
 
grid = GridSearchCV(GradientBoostingClassifier(), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(X_train, y_train)

# print best parameter after tuning
print(grid.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

grid_predictions = grid.predict(X_validation)

# print classification report
metrics_print(y_validation, grid_predictions)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END learning_rate=0.05, loss=deviance, max_depth=3, max_features=log2, n_estimators=10;, score=0.961 total time=   5.9s
[CV 2/5] END learning_rate=0.05, loss=deviance, max_depth=3, max_features=log2, n_estimators=10;, score=0.963 total time=   5.8s
[CV 3/5] END learning_rate=0.05, loss=deviance, max_depth=3, max_features=log2, n_estimators=10;, score=0.965 total time=   7.8s
[CV 4/5] END learning_rate=0.05, loss=deviance, max_depth=3, max_features=log2, n_estimators=10;, score=0.962 total time=   8.6s
[CV 5/5] END learning_rate=0.05, loss=deviance, max_depth=3, max_features=log2, n_estimators=10;, score=0.963 total time=   4.8s
[CV 1/5] END learning_rate=0.05, loss=deviance, max_depth=3, max_features=sqrt, n_estimators=10;, score=0.964 total time=   4.8s
[CV 2/5] END learning_rate=0.05, loss=deviance, max_depth=3, max_features=sqrt, n_estimators=10;, score=0.962 total time=   5.2s
[CV 3/5] END learning_rate=0.05, los

In [ ]:

def neural_net(X_train, y_train, X_validation, y_validation):
    print("NEURAL NET")
    early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=2, mode='auto')

    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy']
                  )

    model.fit(X_train, y_train,
              verbose=0,
              epochs=200,
              batch_size=100,
              callbacks=[early_stop],
              validation_data=(X_validation, y_validation))

    y_pred = np.round(model.predict(X_validation))

    metrics_print(y_validation, y_pred)
    return model


In [ ]:
neural_net(X_train, y_train, X_validation, y_validation)

# Final prediction

In [ ]:
## Final prediction e.g.
model = SVC()
model.fit(X_tot, y_tot)
y_pred = model.predict(X_test)

In [ ]:
#model = XGBClassifier(colsample_bytree=0.6, gamma=0.5, max_depth=5, min_child_weight=5, subsample=0.8)
#model.fit(X_tot, y_tot)
#y_pred = model.predict(X_test)

In [ ]:
y_pred = pd.DataFrame(y_pred, columns=['category'])
y_pred.to_csv('svm.csv', index_label = 'id')